In [2]:
import pandas as pd
df_seoul = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df_seoul[:2]

,번호,분류,제목,내용,내용번호
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045


## 자연어 전처리

In [3]:
stopwords = []

In [8]:
# 토크나이징 함수 정의
from konlpy.tag import Okt
okt = Okt()

def tokenizer(raw, pos=["Noun","Alpha","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [9]:
# tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer)

In [10]:
features = tfidfVectorizer.fit_transform(df_seoul['제목'])
features.toarray()[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 유사도확인

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
def similarity(x):
    result_list = cosine_similarity(features.toarray()[x:x+1], features.toarray())
    df_seoul['유사도'] = result_list[0]
    return df_seoul.sort_values(by='유사도', ascending=False)

In [68]:
similarity(0) # 3번에 대한 유사도 검증

,번호,분류,제목,내용,내용번호,유사도
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,1.000000
1386,1259,행정,육아휴직수당은 얼마인가요?,육아휴직수당은 얼마인가요자녀을 양육하기 위하여 일 이상 휴직한 남녀공무원이 대상이며...,2896657,0.375975
72,2573,복지,"아이조아 아빠교실, 아이조아 놀이교실",아이조아 아빠교실 아이조아 놀이교실 업무개요 다양한 교육과 체...,4369620,0.197549
1757,888,행정,건축과에 무단증축을 민원을 제기했는데 민원인의 인적사항을 요구합니다. 꼭 필요한 것...,건축과에 무단증축을 민원을 제기했는데 민원인의 인적사항을 요구합니다 꼭 필요한 것인...,2895760,0.000000
1758,887,복지,광진구 성폭력 예방교육을 하고 있나요?,광진구 성폭력 예방교육을 하고 있나요광진구청 가정복지과 여성정책팀에서는 매년 학기 ...,2897798,0.000000
...,...,...,...,...,...,...
885,1760,경제,아파트 입주자 등 과반수의 동의를 얻어 개정된 관리규약에 개정시기가 명기되지 않은 ...,아파트 입주자 등 과반수의 동의를 얻어 개정된 관리규약에 개정시기가 명기되지 않은 ...,2896144,0.000000
886,1759,행정,인터넷전화(IPT)시스템 개통일은 언제인가요?,인터넷전화시스템 개통일은 언제인가요중랑구 인터넷전화 개통은 년 월 일입니다,2895585,0.000000
887,1758,경제,성동구치소 이전 ?,성동구치소 이전 성동구치소는 서울동부구치소로 이름을 바꿔 송파구 문정 법조단지 신축...,2896772,0.000000
888,1757,행정,상속등기를 위하여 필요한 사망한 등기명의인의 등록사항별 증명서(‘제적등본’ 포함) ...,상속등기를 위하여 필요한 사망한 등기명의인의 등록사항별 증명서제적등본 포함 및 상속...,2896367,0.000000
